In [33]:
import numpy as np
import pandas as pd
df_ads = pd.read_csv('./dataset/advertising.csv')
df_ads.head()

,wechat,weibo,others,sales
0,304.4,93.6,294.4,9.7
1,1011.9,34.4,398.4,16.7
2,1091.1,32.8,295.2,17.3
3,85.5,173.6,403.2,7.0
4,1047.0,302.4,553.6,22.1


In [34]:
X = np.array(df_ads)
X = np.delete(X, [3], axis=1) # 删除标签
y = np.array(df_ads.sales)
print ("张量X的阶:",X.ndim)
print ("张量X的形状:", X.shape)
print (X)

张量X的阶: 2
张量X的形状: (200, 3)
[[ 304.4   93.6  294.4]
 [1011.9   34.4  398.4]
 [1091.1   32.8  295.2]
 [  85.5  173.6  403.2]
 [1047.   302.4  553.6]
 [ 940.9   41.6  155.2]
 [1277.2  111.2  296. ]
 [  38.2  217.6   16.8]
 [ 342.6  162.4  260. ]
 [ 347.6    6.4  118.4]
 [ 980.1  188.8  460.8]
 [  39.1   16.8    8. ]
 [  39.6  391.2  600. ]
 [ 889.1  381.6  423.2]
 [ 633.8  116.    81.6]
 [ 527.8   61.6  184.8]
 [ 203.4  206.4  164.8]
 [ 499.6  382.4  411.2]
 [ 633.4  114.4  204.8]
 [ 437.7  118.4  311.2]
 [ 334.   136.   103.2]
 [1132.   216.8  183.2]
 [ 841.3  351.2   13.6]
 [ 435.4   11.2   59.2]
 [ 627.4  371.2  472. ]
 [ 599.2  147.2  276.8]
 [ 321.2  128.   326.4]
 [ 571.9  295.2  633.6]
 [ 758.9  336.    28.8]
 [ 799.4  123.2   19.2]
 [ 314.    74.4    7.2]
 [ 108.3  280.8  527.2]
 [ 339.9  395.2  365.6]
 [ 619.7  153.6  132.8]
 [ 227.5   92.8  147.2]
 [ 347.2  220.   128. ]
 [ 774.4   62.4  281.6]
 [1003.3  265.6  303.2]
 [  60.1  127.2  396.8]
 [  88.3  128.   178.4]
 [1280.4  316.

In [35]:
y = y.reshape(-1, 1) # 通过reshape函数把向量转换为矩阵，-1就是len(y),返回样本个数
print ("张量y的形状:", y.shape)

张量y的形状: (200, 1)


In [36]:
# 将数据集进行80%（训练集）和20%（验证集）的分割
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [37]:
# 数据压缩(归一化处理)
def scaler(train, test):
    min = train.min(axis=0)
    max = train.max(axis=0)
    gap = max - min
    train -= min
    train /= gap
    test -= min
    test /= gap
    return train, test

In [38]:
# 用于后面反归一化过程
def min_max_gap(train):
    min = train.min(axis=0)
    max = train.max(axis=0)
    gap = max - min
    return min, max, gap

y_min, y_max, y_gap = min_max_gap(y_train)

In [39]:
X_train_original = X_train.copy() # 保留一份训练集数据副本，用于对要预测数据归一化

In [40]:
X_train, X_test = scaler(X_train, X_test)
y_train, y_test = scaler(y_train, y_test)

In [41]:
x0_train = np.ones((len(X_train), 1))# 构造X_train长度的全1数组配合对Bias的点积
X_train = np.append(x0_train, X_train, axis=1)#把X增加一系列的1
x0_test = np.ones((len(X_test), 1))
X_test = np.append(x0_test, X_test, axis=1)
print ("张量X的形状:", X_train.shape)
print (X_train)

张量X的形状: (160, 4)
[[1.         0.39995488 0.1643002  0.42568162]
 [1.         0.72629521 0.83975659 0.34564644]
 [1.         0.22746071 0.31845842 0.35620053]
 [1.         0.66952402 0.05679513 0.30167106]
 [1.         0.81803143 0.98782961 0.38698329]
 [1.         0.35341003 0.27789047 0.09322779]
 [1.         0.24355215 0.40567951 0.28320141]
 [1.         0.44852996 0.83975659 0.40105541]
 [1.         0.44544703 0.09330629 0.07915567]
 [1.         0.71636965 0.86612576 0.294635  ]
 [1.         0.46597489 0.03245436 0.07651715]
 [1.         0.46319272 0.03651116 0.23131047]
 [1.         0.11594857 0.81135903 0.10202287]
 [1.         0.07353936 0.78498986 0.07915567]
 [1.         0.97706594 0.85192698 0.44766931]
 [1.         0.45770359 0.93509128 0.51627089]
 [1.         0.22204677 0.18255578 0.00527704]
 [1.         0.1898639  0.23732252 0.3764292 ]
 [1.         0.94871795 0.79716024 0.48812665]
 [1.         0.49808256 0.71602434 0.05013193]
 [1.         0.70682006 0.59229209 0.079155

### 3.6.2 多变量的损失函数和梯度下降

In [42]:
# # 手工定义一个MSE均方误差函数,W此时是一个向量
def loss_function(X, y, W):
    y_hat = X.dot(W.T)
    loss = y_hat.reshape((len(y_hat), 1)) - y
    cost = np.sum(loss**2) / (2*len(X))
    return cost

In [43]:
# 定义梯度下降函数
def gradient_descent(X, y, W, alpha, iterations):
    l_history = np.zeros(iterations)
    W_history = np.zeros((iterations, len(W)))
    for iter in range(iterations):
        y_hat = X.dot(W)
        loss = y_hat.reshape((len(y_hat), 1)) - y   # 中间过程, y_hat和y真值的差
        derivative_W = X.T.dot(loss) / (2*len(X))   # 求出多项式的梯度向量
        derivative_W = derivative_W.reshape(len(W))
        W = W- alpha * derivative_W.reshape(len(W))
        l_history[iter] = loss_function(X, y, W)
        W_history[iter] = W
    return l_history, W_history

### 3.6.3 构建一个线性回归函数模型

In [44]:
# 首先确定参数的初始值
iterations = 300
alpha = 0.5
weight = np.array([0.5, 1, 1, 1])
print('当前损失：', loss_function(X_train, y_train, weight))

当前损失： 0.8039183733604858


In [45]:
# 定义线性回归模型
def linear_regression(X, y, weight, alpha, iterations):
    loss_history, weight_history = gradient_descent(X, y, 
                                                        weight, alpha, iterations)
    print("训练最终稿损失：", loss_history[-1])
    y_pred = X.dot(weight_history[-1])
    training_acc = 100 - np.mean(np.abs(y_pred-y))*100  # 计算准确率
    print("线性回归训练准确率：{:.2f}%".format(training_acc))
    return loss_history, weight_history

In [46]:
# 调用刚才定义的线性回归模型
loss_history, weight_history = linear_regression(X_train, y_train, 
                                                                weight, alpha, iterations)

训练最终稿损失： 0.0020534948072892627
线性回归训练准确率：75.99%


In [47]:
print("权重历史记录：", weight_history[-1])
print("损失历史记录：", loss_history[-1])

权重历史记录： [0.05592132 0.64217276 0.22460039 0.0889083 ]
损失历史记录： 0.0020534948072892627


In [48]:
# 预测
X_plan = [250, 50, 50]
X_train, X_plan = scaler(X_train_original, X_plan)  # 归一化
X_plan = np.append([1], X_plan)
y_plan = np.dot(weight_history[-1], X_plan)
y_value = y_plan*y_gap + y_min
print("预测商品的销售额：", y_value, "千元。")

预测商品的销售额： [7.94499196] 千元。
